In [ ]:
from PIL import Image
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# set results_folder to the folder in the same directory with the name "results"
results_folder = '../results/'
train_folder = '../results/train/'
test_folder = '../results/test/'
validation_folder = '../results/val/'

In [ ]:
num_classes = 64

input_shape = (200, 200, 1)

model = Sequential()

layers = [
Conv2D(32, kernel_size=(3, 3), activation="relu"),
MaxPooling2D(pool_size=(2, 2)),
Conv2D(64, kernel_size=(3, 3), activation="relu"),
MaxPooling2D(pool_size=(2, 2)),
Flatten(),
Dropout(0.5),
Dense(num_classes, activation="softmax")
]

for layer in layers:
    model.add(layer)

model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_folder, target_size=(200, 200), batch_size=4000, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(validation_folder, target_size=(200, 200), batch_size=41056, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_folder, target_size=(200, 200), batch_size=4000, class_mode='categorical')

In [ ]:
# Train the model
model.fit(train_generator, steps_per_epoch=47, epochs=30, validation_data=validation_generator, validation_steps=1)

In [ ]:
model.save('first_successful_model.h5')

In [ ]:
# Evaluate the model
score = model.evaluate(test_generator, steps=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Load the model from first_successful_model.h5

from keras.models import load_model
model = load_model('first_successful_model.h5')